## Latent Dirichlet Allocation

+ Most commonly used in natural language processing
+ Sometimes as an end in and of itself
+ Sometimes as a variable reduction technique


### Simple Example of LDA in NLP

Stolen from: http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-topics-extraction-with-nmf-lda-py

+ Authors: 
    + Olivier Grisel <olivier.grisel@ensta.org>
    + Lars Buitinck
    + Chyi-Kwei Yau <chyikwei.yau@gmail.com>
+ License: BSD 3 clause

In [53]:
from __future__ import print_function
from time import time
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups


### This code defines a custom function that we'll use later

In [54]:
n_samples = 2000  #posts / articles
n_features = 1000 # Words
n_topics = 10 
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()



### This code loads the dataset

In [55]:

# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))


Loading dataset...
done in 3.242s.


In [56]:
dataset['data'][20]

u"\n\n\n\n\tI'd like to see this info as well.  As for wavelength, I think\nyou're primarily going to find two - 880 nM +/- a bit, and/or 950 nM\n+/- a bit.  Usually it is about 10 nM either way.  The two most common\nI have seen were 880 and 950 but I have also heard of 890 and 940.\nI'm not sure that the 10 nM one way or another will make a great deal of\ndifference.\n\n\tAnother suggestion - find a brand of TV that uses an IR remote,\nand go look at the SAMS photofact for it.  You can often find some very\ndetailed schematics and parts list for not only the receiver but the\ntransmitter as well, including carrier freq. specs. and tone decoding\nspecs. if the system uses that."

USE term frequancy TF (raw term count) features for LDA

In [84]:
# Use tf (raw term count) features for LDA.

#DF as document frequancy
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))



Extracting tf features for LDA...
done in 0.646s.


In [58]:

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)




Fitting LDA models with tf features, n_samples=2000 and n_features=1000...


In [68]:
t0 = time()
X = lda.fit_transform(tf)
print("done in %0.3fs." % (time() - t0))

done in 50.724s.


In [73]:
X[0]

array([ 0.00344879,  0.20662193,  0.22790615,  0.00344876,  0.00344859,
        0.54133022,  0.00344853,  0.00344855,  0.00344935,  0.00344914])

In [ ]:
print(lda.print_topics(num_topics=10, num_words=3))

In [24]:

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
edu com mail send graphics ftp pub available contact version list ca faq program university cs machines server sun file
Topic #1:
just don like think know good people way ve make right going want use really ll time say sure doesn
Topic #2:
think book new president atheism game pittsburgh history jobs media like subject play time magi people mark just read radio
Topic #3:
drive windows disk use thanks card drives using hard does software scsi problem file 16 help controller pc time need
Topic #4:
hiv health aids april research disease care information medical 1993 national new children study said service university test number drug
Topic #5:
god people does jesus law bible say israel life just believe don church fact think way good know time did
Topic #6:
10 55 11 15 12 game 18 team 20 19 13 period 17 23 16 25 21 24 22 14
Topic #7:
car new year bike price cars good engine oil insurance better just used speed 00 like 000 sell years 100
Topic #8:
people sai

### In class assignment

+ load in the training set (done for you below)
+ re-run LDA and use topics as input for model
+ Predict categories using some multinomial classifier 

In [79]:
print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'), 
                            subset="train")

data = dataset.data

y = dataset.target[:n_samples]

print("done in %0.3fs." % (time() - t0))


Loading dataset...
done in 3.314s.


In [80]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(lda)
print("done in %0.3fs." % (time() - t0))



Extracting tf features for LDA...


TypeError: 'LatentDirichletAllocation' object is not iterable

In [81]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [91]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)
    
model.fit(X, y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [87]:
model.predict(X)


array([17,  0, 17, ...,  3, 15,  9])

In [88]:
model.predict_proba(X)


array([[ 0.3 ,  0.  ,  0.  , ...,  0.4 ,  0.05,  0.  ],
       [ 0.65,  0.  ,  0.  , ...,  0.  ,  0.05,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.6 ,  0.  ,  0.05],
       ..., 
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.05,  0.  ,  0.  , ...,  0.05,  0.  ,  0.1 ],
       [ 0.  ,  0.05,  0.  , ...,  0.  ,  0.  ,  0.  ]])

### In class assignment:

+ I'll divide you into 3 segments
+ Each segment generates 100 sentences on the *same topic*
+ Save as a JSON and send to me
+ We'll run them through LDA

In [ ]:
CLASSLIST = []